### Es 1
Hai a disposizione un file `data.csv` contenente dati mensili di passeggeri con due colonne:

- `date`: data in formato `YYYY-MM` (mese/anno)
- `passengers`: numero di passeggeri per quel mese


Costruisci un modello di **regressione polinomiale** che approssima l’andamento del numero di passeggeri nel tempo.

1. Carica il dataset.
2. Convertilo in un formato numerico utilizzando una colonna `mese_numerico` che conti i mesi a partire da gennaio 1949.
3. Applica una regressione polinomiale (grado a tua scelta).
4. Calcola l’RMSE tra i valori reali e quelli predetti.
5. Visualizza i dati reali e la curva stimata con Plotly.

In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import root_mean_squared_error

df = pd.read_csv('data/data.csv')

df['mese_numerico'] = range(1, len(df['date']) + 1)

X = df[['mese_numerico']].values
y = df[['passengers']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)

modello = make_pipeline(PolynomialFeatures(2), LinearRegression())
modello.fit(X_train, y_train)
y_pred = modello.predict(X_test)

mask_validi = ~np.isnan(y_test.ravel())

print("RMSE: {}".format(root_mean_squared_error(y_test[mask_validi], y_pred[mask_validi])))

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['mese_numerico'], y=df['passengers'], mode='markers', name='Dati reali'))

fig.add_trace(go.Scatter(x=X_test[:, 0], y=y_pred[:, 0], mode='lines', name="Regressione"))

fig.update_layout(
    title='Passeggeri nel tempo con curva di regressione',
    xaxis_title='Mese',
    yaxis_title='Passeggeri'
)

fig.show()

RMSE: 125.67636762056098


### Es2. 
Costruisci una web app con Dash che permette all’utente di scegliere il grado del polinomio per adattare un modello di regressione ai dati non lineari e vedere il risultato aggiornarsi dinamicamente.


1. Genera 100 punti x tra -3 e 3.

2. Calcola ad esempio y = x³ - x + rumore.

3. Costruisci un'interfaccia Dash con:
    - uno slider per scegliere il grado del polinomio (1–10),
    - un grafico Plotly che mostra i dati e la curva stimata.

4. Usa PolynomialFeatures + LinearRegression da scikit-learn per stimare la curva

In [1]:
import numpy as np
import plotly.graph_objects as go
import dash                      
from dash import dcc, html      
from dash.dependencies import Input, Output
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

punti_x = np.random.uniform(-3, 3, 100)
rumore_dove = np.random.randint(0, 100, 25) #prendo 25 indici a caso in cui inserire l'errore

punti_y = punti_x**3 - punti_x

for i in rumore_dove:
    punti_y[i] += np.random.uniform(-1.5, 1.5, 1)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H3("Seleziona un numero"),

    #slider da 1 a 10 con step 1 e valore iniziale 2
    dcc.Slider(
        id='slider-numero',
        min=1,
        max=10,
        step=1,
        value=2, #valore iniziale
        marks={i: str(i) for i in range(1, 11)},
        updatemode='drag'
    ),

    dcc.Graph(id='regression_plot')
])

@app.callback(
    Output('regression_plot', 'figure'), 
    Input('slider-numero', 'value') 
)
def aggiorna_grafico(valore):
    X_train, X_test, y_train, y_test = train_test_split(punti_x, punti_y, test_size=0.3, shuffle=False)

    X_train = X_train.reshape(-1, 1)
    X_test = X_test.reshape(-1, 1)

    modello = make_pipeline(PolynomialFeatures(valore), LinearRegression())
    modello.fit(X_train, y_train)

    x_curve = np.linspace(punti_x.min(), punti_x.max(), 300).reshape(-1, 1)
    y_curve = modello.predict(x_curve)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=punti_x, y=punti_y, mode='markers', name='Dati reali'))
    fig.add_trace(go.Scatter(x=x_curve.ravel(), y=y_curve, mode='lines', name="Curva di regressione"))

    fig.update_layout(
        title='Curva di regressione',
        xaxis_title='X',
        yaxis_title='Y'
    )

    return fig


if __name__ == '__main__':
    app.run_server(debug=True)

C:\Users\grayr\AppData\Local\Temp\ipykernel_4252\4017762166.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  punti_y[i] += np.random.uniform(-1.5, 1.5, 1)
